In [1]:
import sys
sys.path.append('D:\github\python')
import pandas as pd
import xlsxwriter
import re
import time
import numpy as np
# from Modul.modulku import praproses as pps
from Cek_typo import cek_typo as ct
from Normalisasi_KBBI import normalisasi_kbbi as nkbi
from modulku import praproses as pps
from modulku import StemNstopW as stm
import re
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

In [8]:
data = pd.read_excel("data/data_train_B.xlsx")
respon = data['RESPONSE']
res_id = data['RES_ID'].tolist()
label = data['LABEL']
# label = [int(x) for x in label]
# respon = [x.lower() for x in respon]
data.head()

,RES_ID,LABEL,label2,RESPONSE
0,TRB48,1.0,0,upaya untuk membuat produksi pakaian menjadi l...
1,TRB185,0.0,0,upaya untuk membuat produksi pakaian menjadi l...
2,TRB100,0.0,0,untuk meningkatkan kualitas barang yang memuas...
3,TRB82,0.0,0,untuk mengurangi penggunan air
4,TRB37,0.0,0,untuk membuat toko tersebut lebih bagus lagi d...


In [3]:
respon_list = list()
ixx = 1
oo = 0
for teks in respon:
    a = tknzr.tokenize(teks)
    a = " ".join(a)
#     a = teks
    a = pps.preprocessing(teks)
    a = pps.removePunc(a)
    a = ct.norm_typo(a)
    a = nkbi.norm_kbbi(a)
    a = stm.stemmer_kata(a)
#     a = stm.stop_word(a)
    a = re.sub(' +', ' ',a)
    a = a.lstrip()
    respon_list.append(a)
    if ixx%10==0:
        #oo+=1
        print(".", end ="")
    if ixx%50==0:
        print(ixx, end="")
    ixx+=1

.....50.....100.....150.....200.....250.....300

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(respon_list)

fitur = vectorizer.get_feature_names()

In [6]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
new_labels =  kmeans.labels_
y = new_labels
len(y)

305

In [9]:
dit_ ={
    'res-id':res_id,
    'label':label,
    'new_label': y,
    'respon_old':respon,
    'respon':respon_list
}
h = pd.DataFrame.from_dict(dit_)
h.to_excel('data/Data_B_Bersih_2.xlsx')
h.head()

,res-id,label,new_label,respon_old,respon
0,TRB48,1.0,1,upaya untuk membuat produksi pakaian menjadi l...,upaya untuk buat produksi pakai jadi lebih etika
1,TRB185,0.0,1,upaya untuk membuat produksi pakaian menjadi l...,upaya untuk buat produksi pakai jadi lebih etika
2,TRB100,0.0,1,untuk meningkatkan kualitas barang yang memuas...,untuk tingkat kualitas barang yang muas untuk ...
3,TRB82,0.0,0,untuk mengurangi penggunan air,untuk kurang guna air
4,TRB37,0.0,0,untuk membuat toko tersebut lebih bagus lagi d...,untuk buat toko sebut lebih bagus lagi dan leb...
